# 検証用 Fact-Checker
チームみらいが公開しているFact-CheckerからAI関連の処理のみを抽出して、動作を検証しやすいようにいpythonで実行できるようにしました。  
Fact-Checkerをコマンドラインで動かすのと同様の動作で、入力されたテキストに対して判定を行います。  

## 準備
OpenAIのAPIキーに加えて、任意の参照したい資料をアップロードしたベクトルストアのIDを用意しておいてください。  
ベクトルストアについては、APIプラットフォームにログインして、Dashboard > Storage からアップロードするのが手軽かと思います。  
openaiのライブラリはあまり古すぎなければ実行可能かと思いますが、エラーとなるような場合はライブラリをアップデートしてください。  
以下で動かした際のバージョンは1.82.0です。

In [1]:
# %pip install openai

from openai import OpenAI

In [ ]:
# .envを使用してAPIキーを読み込む場合
# from dotenv import load_dotenv
# load_dotenv()

client = OpenAI()

# ベクトルストアのIDを指定
vectorStoreId = "your_vector_store_id"

In [3]:
def fact_check(statement: str) -> str:
    """
    ファクトチェックを行う関数

    :param statement: チェック対象の文
    :return: ファクトチェックの結果
    """
    if not statement:
        return "❌ 文章を引数で渡してください。例:\n fact_check('地球は平らである')"


    response = client.responses.create(
        model= "o3-mini",
        tools=[{
            "type": "file_search",
            "vector_store_ids": [vectorStoreId],
        }],
        include=["file_search_call.results"],
        input=[
            {
                "role": "system",
			"content": """あなたは厳格なファクトチェッカーです。  
以下の手順と書式だけを守り、日本語で簡潔に回答してください。  
（指示にないことは書かないこと）

────────────────────────────────
▼ステップ 0 : 対象判定（事前フィルタ）
  ❶ 入力テキストが「客観的に検証可能な事実命題」か確認せよ。
  ❷ 以下のいずれかに該当する場合はファクトチェック対象外とし、  
      次の書式で即座に終了すること：
        OK
        入力文は○○のためファクトチェック対象外。
      （○○には一行で理由を書く。出典は不要）

  ★ファクトチェック対象外リスト
    ・感想／意見／価値判断／予測／願望／比喩／誇張  
    ・固有名詞そのもの  
      （人名・地名・組織名・商品名・ブランド名 等）  
    ・連絡先や識別情報（URL, メールアドレス, 電話番号, SNS ID 等）  
    ・個人の経歴・肩書・受賞歴など履歴情報  
    ・検証可能な公開データソースが存在しない内容

────────────────────────────────
▼ステップ 1 : 真偽判定（ステップ 0 を通過した場合のみ）
  ❶ データソースで裏付けを取り、最上部に以下いずれかを記載
        OK : データソースと完全一致  
        NG : データソースと矛盾（誤りあり）  
        OK : データ不足で判定不能  

  ❷ 判定根拠を箇条書き（簡潔に）。  
  ❸ 引用箇所（節・ページ・タイムスタンプ等）を箇条書き。  
  ❹ 最後に出典（URL／書誌情報）。

  ★追加ルール
    ・表記揺れ（漢字⇔ひらがな、略称、旧字体など）による  
      固有名詞の差異は誤りとみなさない。  
      ─ 例：「安野貴博」と「安野たかひろ」は同一人物扱い。  
      ─ 誤字脱字のみを指摘する用途ではないことに注意。  
    ・固有名詞の spelling が異なること「だけ」を理由に  
      NG 判定を出さない。内容面の食い違いがある場合のみ NG とする。

────────────────────────────────
▼出力フォーマット例

OK
- 根拠: …  
- 該当箇所: …  
- 出典: …

NG
- 誤り: …  
- 正しい情報: …  
- 出典: …

OK
入力文は主観的感想であり客観的事実ではないため。
────────────────────────────────
        """,
                },
                {
                    "role": "user",
                    "content": statement,
                },
            ],
    )

    return response.output_text

## 検証例

最新のバージョンでは削除されていますが、おそらく検証用に使われたと思われる[ファイル](
https://github.com/team-mirai/fact-checker/blob/15718cf7926685f0c5f15ad2fbf056009fb7a2a6/manifest.md)が以前は公開されていたので、このファイルをベクトルストアにアップロードして、主な動作を確認してみます。
Temperatureなどのパラメータの指定はされていないので、実行によって以下とは異なる回答・結果となる可能性はあります。  
実行時のo3-miniのバージョンはo3-mini-2025-01-31です。  

４つ目の「データソースに十分な情報がなく判定不能なためOK」のケースは、参照するファイルが１ファイルのみなことや聞き方にも影響されると思いますが、データソースの記載とは矛盾しておらず参照するファイルに記述がない、というケースではNGと判定されることが多いかもしれません。  

検知されたものは人の手で確認を行うということで、誤検知(本来OKのものをNGと判例する)については、システム上それほど問題はないものかと思います。

In [ ]:
 # 主観的な感想なのでOK
print(fact_check("今日は楽しかったです。") )

 # データソースと一致しているのでOK
print(fact_check("チームみらいのマニフェストは作成途中版らしい") )

 # データソースと矛盾しているのでNG
print(fact_check("チームみらいのマニュフェストは５月末でもう完成している") )

 # データソースに十分な情報がなく(検証用に使用したファイルには公開しているツール・システムなどの記載はなし)、判定不能なためOK (結果ではMGと判定)
print(fact_check("チームみらいはファクトチェック用のシステムを公開している") )

OK
入力文は主観的感想であり客観的事実ではないため。
OK
- 根拠: アップロードされたmanifest.mdに「本マニフェストは作成途中版であり、内容は今後変更の可能性があります」と明記されているため。
- 該当箇所: manifest.mdの冒頭部分（）。
- 出典: manifest.md
NG
- 誤り: マニュフェストは作成途中であり、意見を取り入れて改善中の状態で、５月末に完成しているとの記述は確認できません。
- 正しい情報: マニュフェストはバージョン0.1であり、多くの箇所が議論中・作業中であると記載されており、完成状態ではなく、今後変更の可能性があります。 
NG
- 誤り: チームみらいが公開しているのは、政策に関するマニフェストや議論を促すための「AI熟議システム『いどばた』」などであり、ファクトチェック専用のシステムを公開しているとの記述は確認できません。
- 正しい情報: 現時点の公開文書（マニフェスト等）には、ファクトチェック用のシステム公開に関する情報は含まれていません .
